In [1]:
# Achtung normalize und invert sind etwas anders weil pos schon auf 0,1
# Alle Trajektorien sind in einem file für die 6 input arten
# 900 Traj mit je 500 zeitschritten
# die Testtrajektorien sind immer die Enden der 30 Trajektorien (6*5) und 7mal länger

# Vorher T  = [0 , 0.0112] 500 steps
# Jetzt T = [0, 0.0224] 500 steps 
# => Schrittweite verdoppelt    
# Modell normal trainieren und dann bei Vergleich zu Simulink die Simulink Daten interpolieren damit sie gleich lang sind!

In [2]:
from meas_get_data import *
from meas_test_func_fs import * 
from meas_get_data import *
import os
torch.set_default_dtype(torch.float64)
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
path = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
data =get_data(path, num_inits=0)
data.size()

path_test_data = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\Test-Messdaten_30traj_7times500steps.csv"
test_data =get_data(path_test_data, num_inits=0)
test_data.size()

path_test_data_simulink = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\Messdaten_Simulink_Vergleich.csv"
test_data_simulink =get_data(path_test_data_simulink, num_inits=0)
test_data_simulink.size()

print("data size :", data.size())
print("test_data size: ", test_data.size())
print("test_data_simulink size: ", test_data_simulink.size())

data size : torch.Size([908, 500, 4])
test_data size:  torch.Size([30, 3500, 4])
test_data_simulink size:  torch.Size([30, 3500, 4])


In [45]:

for i in range(1):
  visualise(normalize_invert(test_data_simulink),i)
  visualise(normalize_invert(test_data),i)




In [ ]:
visualise(normalize_invert(data),89)
visualise(normalize_invert(data),91)
visualise(normalize_invert(data),92)

In [23]:
%matplotlib inline

In [25]:
import torch
from meas_get_data import *
from meas_NN_classes import *
from meas_dataloader_fs import *
from meas_test_func_fs import *

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def plot_results(x, pred, rescale=False, window_size=1, sim_data_index=0):
    
    if rescale:
        x = normalize_invert(x)
        pred = normalize_invert(pred)

    if x.dim() == 3:
        x = x.view(x.size(dim=1), x.size(dim=2))
    if pred.dim() == 3:
        pred = pred.view(pred.size(dim=1), pred.size(dim=2))



    figure , axs = plt.subplots(4,1, figsize=(9,9))
    figure.tight_layout(pad=2.0)

    stepsize = 2 * 2e-5 
    time = np.linspace(0,x.size(dim=0)* stepsize, x.size(dim=0))

    axs[0].plot(time, x.detach().cpu().numpy()[:, 0], color="darkgreen", label="data")
    axs[0].set_title("NC : Input Voltage 1")
    axs[0].set_ylabel("[V]")

    axs[1].plot(time, x.detach().cpu().numpy()[:, 1], color="darkgreen", label="data")
    axs[1].set_title("NO : Input Voltage 2")
    axs[1].set_ylabel("[V]")

    axs[2].plot(time, pred.detach().cpu().numpy()[:, 2], color="red", label="pred")
    axs[2].plot(time, x.detach().cpu().numpy()[:, 2], color="blue", label="data", linestyle="dashed")
    axs[2].set_title("pressure")
    axs[2].set_ylabel("[Pa]")

    axs[3].plot(time, pred.detach().cpu().numpy()[:, 3], color="red", label="pred")
    axs[3].plot(time, x.detach().cpu().numpy()[:, 3], color="blue", label="data", linestyle="dashed")
    axs[3].set_title("position")
    axs[3].set_ylabel("[m]")
    axs[3].set_xlabel(f"time [s]")

    axs[2].axvline(x=time[window_size], color='black', linestyle='--', label='start of prediction')
    axs[3].axvline(x=time[window_size], color='black', linestyle='--', label='start of prediction')


    if sim_data_index >= 0:
        path_test_data_simulink = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\Messdaten_Simulink_Vergleich.csv"
        test_data_simulink = get_data(path_test_data_simulink, num_inits=0)
        #test_data_simulink[id,:,2]
        axs[2].plot(time, test_data_simulink[sim_data_index,:,2], color="orange", label="pressure_simulink")
        axs[3].plot(time, test_data_simulink[sim_data_index,:,3], color="orange", label="position_simulink")


    if rescale:
        u1_max = 200  #Spannung in [V]              ... [0, 200]
        u1_min = 0
        u2_max = 200
        u2_min = 0
        p_max = 3.5*1e5 #Druck in [bar]             ... [1, 3.5]
        p_min = 1.0*1e5 #Umgebungsdruck in [bar]
        s_max = 0.605*1e-3     #Position [m]          ... [0, 0.0006]
        s_min = 0.0

        axs[0].set_ylim(u1_min-10, u1_max+10)
        axs[1].set_ylim(u2_min-10, u2_max+10)
        axs[2].set_ylim(p_min-0.1*p_max, p_max+0.1*p_max)
        axs[3].set_ylim(s_min-+0.1*s_max, s_max+0.1*s_max)

    for i in range(4):
        axs[i].grid(True)
        axs[i].legend()



    plt.grid(True)
    plt.legend()
    plt.show()

def test(data, model, model_type="lstm", window_size=1 ,display_plots=False, numb_of_inits=1, fix_random=True, rescale=False, specific_index=-1):

    if fix_random:
     np.random.seed(1234)
    else:
     np.random.seed(seed=None)
      
    test_inits = data.size(dim=0)
    ids = np.random.choice(test_inits, min([numb_of_inits, test_inits]), replace=False)
    ids = np.unique(ids)

    

    if specific_index >= 0:
        data = data[specific_index:specific_index+1,:, :]
    else :
        data = data[ids,:, :]   
    loss_fn = nn.MSELoss()
    timesteps = data.size(dim=1)

    total_loss = 0
    for i, x in enumerate(data):

        with torch.inference_mode():
            if model_type == "lstm":
                x=x.to(device)        
                x = x.view(1,x.size(dim=0), x.size(dim=1))                
                pred = torch.zeros((timesteps, 4), device=device)

                pred[0:window_size, :] = x[0, 0:window_size, :]
                pred[:, 0] = x[0, :, 0]

                x_test = x.clone()
                x_test[:,window_size:,2:] = 0
                x_test = x_test.to(device)

                out, _ = model(x_test) 
                pred[window_size:,2:] = out

                total_loss += loss_fn(pred[window_size:, 2:], x[0, window_size:, 2:]).detach().cpu().numpy()

                if display_plots:
                    if specific_index>=0:
                        plot_results(x, pred, rescale=rescale, window_size=window_size, sim_data_index=specific_index)
                    else:
                        plot_results(x, pred, rescale=rescale, window_size=window_size, sim_data_index=ids[i])

    return total_loss/data.size(0)

In [11]:
%matplotlib qt

In [32]:

params_lstm =    {
                        "window_size" : 16,
                        "h_size" : 8,
                        "l_num" : 3,
                        "learning_rate" : 0.001,
                        "batch_size" : 20,

                        "percentage_of_data" : 0.8,
                        "cut_off_timesteps" : 0,
                        "part_of_data" : 0,
                        "epochs" : 1000,
                        "test_every_epochs" : 100,

                        "T_max" : 500,

                        "experiment_number" : np.random.randint(0,1000)
                        }


if os.name == "nt":
    path_train_data=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\messdaten_900traj_500steps.csv"
    path_test_data = r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\Messungen\TEST-Messdaten_30traj_7times500steps.csv"

else:
    path_train_data=r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/messdaten_900traj_500steps.csv"
    path_test_data = r"/home/rdpusr/Documents/ventil_lstm/Experiment_Meassurements/Messungen/TEST-Messdaten_30traj_7times500steps.csv"

test_data = get_data(path_test_data,num_inits=0)
train_data = get_data(path_train_data,num_inits=0)
print(test_data.size(), train_data.size())
 

model_lstm = OR_LSTM(input_size=4, hidden_size=params_lstm["h_size"], out_size=2, layers=params_lstm["l_num"], window_size=params_lstm["window_size"]).to(device)

path_lstm=r"C:\Users\strasserp\Documents\ventil_lstm\Experiment_Meassurements\or_lstm_best.pth"
model_lstm.load_state_dict(torch.load(path_lstm, map_location=torch.device(device)))

#test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 1, fix_random=True)
test(test_data.to(device), model_lstm, model_type="lstm", window_size=params_lstm["window_size"], display_plots=True, numb_of_inits = 1, fix_random=True, specific_index=26)

torch.Size([30, 3500, 4]) torch.Size([908, 500, 4])


0.0011690036922297382

In [ ]:
import platform
import psutil
import GPUtil
import os

def get_system_info():

    print("===== Packages - INFORMATION =====")
    
    print("===== SYSTEM INFORMATION =====")
    print(f"System: {platform.system()}")
    print(f"Node Name: {platform.node()}")
    print(f"Release: {platform.release()}")
    print(f"Version: {platform.version()}")
    print(f"Machine: {platform.machine()}")
    print(f"Processor: {platform.processor()}")
    print(f"Python Version: {platform.python_version()}")
    print(f"Architecture: {' '.join(platform.architecture())}")
    print(f"Hostname: {platform.node()}")
    print()

def get_memory_info():
    print("===== MEMORY INFORMATION =====")
    svmem = psutil.virtual_memory()

    print(f"Memory Percentage: {svmem.percent}%")
    print()

def get_cpu_info():
    print("===== CPU INFORMATION =====")
    print(f"Physical cores: {psutil.cpu_count(logical=False)}")
    print(f"Total cores: {psutil.cpu_count(logical=True)}")
    print(f"Max Frequency: {psutil.cpu_freq().max:.2f}Mhz")
    print(f"Min Frequency: {psutil.cpu_freq().min:.2f}Mhz")
    print(f"Current Frequency: {psutil.cpu_freq().current:.2f}Mhz")
    print(f"CPU Usage Per Core: {psutil.cpu_percent(percpu=True)}")
    print(f"Total CPU Usage: {psutil.cpu_percent()}%")
    print()

def get_disk_info():
    print("===== DISK INFORMATION =====")
    partitions = psutil.disk_partitions()
    for partition in partitions:
        print(f"Device: {partition.device}")
        print(f"  Mountpoint: {partition.mountpoint}")
        print(f"  File system type: {partition.fstype}")
        partition_usage = psutil.disk_usage(partition.mountpoint)

        print(f"  Percentage: {partition_usage.percent}%")
    print()

def get_gpu_info():
    print("===== GPU INFORMATION =====")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU Name: {gpu.name}")
        print(f"GPU Load: {gpu.load*100}%")

        print(f"GPU Temperature: {gpu.temperature} °C")
        print(f"GPU Driver Version: {gpu.driver}")
        print()


if __name__ == "__main__":
    get_system_info()
    get_cpu_info()
    get_memory_info()
    get_disk_info()
    get_gpu_info()
